In [1]:
import numpy as np
import pandas as pd
import math
import codecs
import warnings
warnings.filterwarnings('ignore')

二項ロジットモデル：選択肢が二つの場合の離散選択モデル

# 二項ロジットモデル

問題設定：自転車と鉄道の選択

確定項： 
* 定数（LOS変数，SE変数以外の選択肢固有の特性がもたらす効用を表した定数項．選択には選択肢間の差だけが意味を持つので，基準とする選択肢の定数は0にする．今回は自転車を基準とした）
    * 鉄道
* LOS変数（各選択肢独自のサービスレベルを表す）
    * 所要時間
    * 料金
* SE変数（社会経済変数．意思決定者の属性やトリップの属性を表す）

誤差項：  
* ガンベル分布
    * 確定部分に含まれる変数以外の要因 (抜け落ちた変数)
    * 確定部分を線形和とした関数形の誤差
    * 説明変数の測定誤差
    * 行動者の説明変数の認知誤差

効用関数  
$$U = V + \epsilon$$
$$U = \sum \beta x + \epsilon$$
$$U = \beta_p x_{price} + \beta_t x_{time} + \alpha + \epsilon$$

パラメータ推定：  
* 求めるパラメータは３つ
* 推定方法
    * 最尤推定
    * 最適化：ニュートン法

## 実装

### データセット

In [4]:
with codecs.open("../data/BinN/ensyu.csv", "r", "Shift-JIS", "ignore") as file:
    data = pd.read_table(file, delimiter=",")

In [5]:
data.head()

,トリップID,モニターID,目的コード,目的,出発日時,到着日時,実トリップ時間_秒,OD直線距離_m,LatO,LonO,...,所要時間walk,代替手段生成可否bus,距離bus,所要時間bus,費用bus,アクセス距離bus,アクセス時間bus,イグレス距離bus,イグレス時間bus,総所要時間bus
0,255461,yd021,800,散歩･回遊,2009/10/31 06:19:00,2009/10/31 07:50:01,5461,19341.21308,35.345539,139.486467,...,270.4,0,0.0,0,0,0,0.0,0,0.0,0.0
1,256674,yd027,500,買い物,2009/11/13 19:08:13,2009/11/13 20:13:43,3930,20793.13521,35.443606,139.637749,...,293.1,0,0.0,0,0,0,0.0,0,0.0,0.0
2,255291,yd025,400,業務,2009/10/29 07:36:03,2009/10/29 08:21:20,2717,10421.32431,35.366739,139.571525,...,143.4,0,0.0,0,0,0,0.0,0,0.0,0.0
3,256382,yd021,100,出勤,2009/11/10 06:55:33,2009/11/10 07:58:02,3749,24456.99597,35.325508,139.409106,...,342.3,0,0.0,0,0,0,0.0,0,0.0,0.0
4,257459,yd028,700,娯楽,2009/11/21 10:40:54,2009/11/21 12:31:44,6650,42660.90181,35.464259,139.530895,...,595.9,0,0.0,0,0,0,0.0,0,0.0,0.0


In [7]:
lines = []
for r in data:
    lines.append(''.join(r))
print ('\n'.join(lines))

トリップID
モニターID
目的コード
目的
出発日時
到着日時
実トリップ時間_秒
OD直線距離_m
LatO
LonO
shikucodeO
shikunameO
zone4codeO
zone4nameO
zone5codeO
zone5nameO
LatD
LonD
shikucodeD
shikunameD
zone4codeD
zone4nameD
zone5codeD
zone5nameD
年齢
性別コード
性別
自宅位置
代表交通手段コード
代表交通手段
選択結果時間
代替手段生成可否car
距離car
所要時間car
代替手段生成可否train
所要時間train
費用train
出発駅
到着駅
乗換回数train
アクセス距離train
アクセス時間train
イグレス距離train
イグレス時間train
総所要時間train
代替手段生成可否bike
距離bike
所要時間bike
代替手段生成可否walk
距離walk
所要時間walk
代替手段生成可否bus
距離bus
所要時間bus
費用bus
アクセス距離bus
アクセス時間bus
イグレス距離bus
イグレス時間bus
総所要時間bus


In [8]:
data['代表交通手段'].unique()

array(['自転車', '鉄道', '自動車', '徒歩', 'バス'], dtype=object)

In [55]:
len(data[data['代表交通手段'] == '自転車']), len(data[data['代表交通手段'] == '鉄道']), len(data[data['代表交通手段'] == '自動車']), len(data[data['代表交通手段'] == '徒歩']), len(data[data['代表交通手段'] == 'バス'])

(51, 131, 146, 60, 12)

In [6]:
len(data)

400

In [138]:
print('自転車，鉄道，自動車，徒歩が選択肢：', len(data[data['代替手段生成可否bike'] == 1][data['代替手段生成可否train'] == 1][data['代替手段生成可否car'] == 1][data['代替手段生成可否walk'] == 1]))
print('全てを選択肢として持っているtrip数:', len(data[data['代替手段生成可否bike'] == 1][data['代替手段生成可否train'] == 1][data['代替手段生成可否car'] == 1][data['代替手段生成可否walk'] == 1][data['代替手段生成可否bus'] == 1]))

自転車，鉄道，自動車，徒歩が選択肢： 347
全てを選択肢として持っているtrip数: 91


In [152]:
df = data[(data['代表交通手段'] == '自転車') | (data['代表交通手段'] == '鉄道')].reset_index(drop=True)
df = df[(df['代替手段生成可否bike'] == 1) & (df['代替手段生成可否train'] == 1)].reset_index(drop=True)
df = df.loc[:,['代表交通手段', '所要時間bike', '総所要時間train', '費用bike', '費用train']]
df['定数train'] = 1
y = []
for i in range(len(df)):
    if df.loc[i, '代表交通手段'] == '自転車':
        y.append(1)
    else:
        y.append(0)
df['y'] = y

In [153]:
len(df), len(df[df['代表交通手段'] == '自転車']), len(df[df['代表交通手段'] == '鉄道'])

(175, 44, 131)

In [154]:
df.head(30)

,代表交通手段,所要時間bike,総所要時間train,費用bike,費用train,定数train,y
0,鉄道,117.9,55.3,NaN,440,1,0
1,鉄道,58.0,38.6,NaN,210,1,0
2,鉄道,138.3,49.8,NaN,570,1,0
3,鉄道,241.1,76.2,NaN,3660,1,0
4,鉄道,4.5,11.2,NaN,200,1,0
5,鉄道,117.2,68.5,NaN,550,1,0
6,鉄道,13.0,21.5,NaN,180,1,0
7,鉄道,113.8,94.9,NaN,420,1,0
8,鉄道,65.4,45.8,NaN,400,1,0
9,鉄道,59.3,35.6,NaN,210,1,0


### パラメータ推定

In [155]:
# log-likelihood

In [156]:
# Newton
def 